--- Day 3: No Matter How You Slice It ---

The Elves managed to locate the chimney-squeeze prototype fabric for Santa's suit (thanks to someone who helpfully wrote its box IDs on the wall of the warehouse in the middle of the night). Unfortunately, anomalies are still affecting them - nobody can even agree on how to cut the fabric.

The whole piece of fabric they're working on is a very large square - at least 1000 inches on each side.

Each Elf has made a claim about which area of fabric would be ideal for Santa's suit. All claims have an ID and consist of a single rectangle with edges parallel to the edges of the fabric. Each claim's rectangle is defined as follows:

    The number of inches between the left edge of the fabric and the left edge of the rectangle.
    The number of inches between the top edge of the fabric and the top edge of the rectangle.
    The width of the rectangle in inches.
    The height of the rectangle in inches.

A claim like #123 @ 3,2: 5x4 means that claim ID 123 specifies a rectangle 3 inches from the left edge, 2 inches from the top edge, 5 inches wide, and 4 inches tall. Visually, it claims the square inches of fabric represented by # (and ignores the square inches of fabric represented by .) in the diagram below:

...........
...........
...#####...
...#####...
...#####...
...#####...
...........
...........
...........

The problem is that many of the claims overlap, causing two or more claims to cover part of the same areas. For example, consider the following claims:

#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2

Visually, these claim the following areas:

........
...2222.
...2222.
.11XX22.
.11XX22.
.111133.
.111133.
........

The four square inches marked with X are claimed by both 1 and 2. (Claim 3, while adjacent to the others, does not overlap either of them.)

If the Elves all proceed with their own plans, none of them will have enough fabric. How many square inches of fabric are within two or more claims?


In [56]:
import re

class Day3a:
    def __init__(self):
        self.lines = []
        
    def parse_line(self, line):
        # #1 @ 483,830: 24x18
        pattern = r"^#(?P<index>\d+) @ (?P<left>\d+),(?P<top>\d+): (?P<width>\d+)x(?P<height>\d+)"
        
        m = re.search(pattern, line)
        
        if m is None:
            return 0
        
        info = {'line': line}
        for k in ['width', 'height', 'index', 'left', 'top']:
            info[k] = int(m.group(k))
            
        info['right'] = info['left'] + info['width']
        info['bottom'] = info['top'] + info['height']        
        return info
    
    # find the intersection area of two rectangles
    def intersection_rect(self, r1, r2):
        x5 = max(r1['left'], r2['left'])
        y5 = max(r1['top'], r2['top'])
        x6 = min(r1['right'], r2['right'])
        y6 = min(r1['bottom'], r2['bottom'])
        
        return {'left': x5, 'right': x6, 'top': y5, 'bottom': y6, 'width': x6-x5, 'height': y6-y5}

    def is_intersection_valid(self, r):
        if r['height'] < 1: return False
        if r['width'] < 1: return False
        return True
    
    def get_area(self, r):
        return r['width'] * r['height']
    
    def test_rects(self):
        r1 = {'left': 1, 'right': 5, 'top': 3, 'width': 4, 'height': 4, 'bottom': 7}
        r2 = {'left': 3, 'right': 7, 'top': 1, 'width': 4, 'height': 4, 'bottom': 5}
        return [r1, r2]
        
        
    def test_intersection(self):
        #1 @ 1,3: 4x4
        #2 @ 3,1: 4x4
        r1, r2 = self.test_rects()
        r = self.intersection_rect(r1, r2)
        print("test:", r)
        assert r == {'left': 3, 'right': 5, 'top': 3, 'bottom': 5, 'width': 2, 'height': 2}
        
        r1 = {'left': 1, 'right': 2, 'top': 1, 'bottom': 2}
        r2 = {'left': 10, 'right': 11, 'top': 10, 'bottom': 11}
        r = self.intersection_rect(r1, r2)
        print("test:", r)
        
        
    def load(self):
        with open('day3.input') as f:
            self.lines = [ line.strip() for line in list(f) ]
            self.entries = [ self.parse_line(line) for line in self.lines ]

    
    def dimensions(self):
        width = 0
        height = 0
        for i in self.entries:
            width = max(i['right'], width)
            height = max(i['bottom'], height)
        return [width, height]
    
    def setup_fabric(self):
        width, height = self.dimensions()

        self.fabric = [[0 for y in range(height + 1)] for x in range(width + 1)]

    def paint_fabric(self, r):
        for x in range(r['left'], r['right']):
            for y in range(r['top'], r['bottom']):
                try:
                    self.fabric[x][y] += 1
                except:
                    print("problem painting fabric x", x, "y", y)
                
    def count_paint(self):
        total = 0
        for x in self.fabric:
            for count in x:                
                if count <= 1: continue
                total += 1
        return total
        
    def run(self):
        self.load()

        self.test_intersection()
        total_area = 0
        
        
        self.fabric = [[0 for y in range(11)] for x in range(11)]
        for r in self.test_rects():
            self.paint_fabric(r)
        print(self.fabric)
        
        self.setup_fabric()
        for r in self.entries:
            self.paint_fabric(r)
        
                
        return self.count_paint()
                
        
if __name__ == "__main__":
    result = Day3a().run()
    print(result)
    # Answer: 119551

test: {'left': 3, 'right': 5, 'top': 3, 'bottom': 5, 'width': 2, 'height': 2}
test: {'left': 10, 'right': 2, 'top': 10, 'bottom': 2, 'width': -8, 'height': -8}
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 1, 2, 2, 1, 1, 0, 0, 0, 0], [0, 1, 1, 2, 2, 1, 1, 0, 0, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
119551


In [3]:
import re

class Day3b:
    def __init__(self):
        self.lines = []
        
    def parse_line(self, line):
        # #1 @ 483,830: 24x18
        pattern = r"^#(?P<index>\d+) @ (?P<left>\d+),(?P<top>\d+): (?P<width>\d+)x(?P<height>\d+)"
        
        m = re.search(pattern, line)
        
        if m is None:
            return 0
        
        info = {'line': line}
        for k in ['width', 'height', 'index', 'left', 'top']:
            info[k] = int(m.group(k))
            
        info['right'] = info['left'] + info['width']
        info['bottom'] = info['top'] + info['height']        
        return info
    
    # find the intersection area of two rectangles
    def intersection_rect(self, r1, r2):
        x5 = max(r1['left'], r2['left'])
        y5 = max(r1['top'], r2['top'])
        x6 = min(r1['right'], r2['right'])
        y6 = min(r1['bottom'], r2['bottom'])
        
        return {'left': x5, 'right': x6, 'top': y5, 'bottom': y6, 'width': x6-x5, 'height': y6-y5}

    def is_intersection_valid(self, r):
        if r['height'] < 1: return False
        if r['width'] < 1: return False
        return True
    
    def get_area(self, r):
        return r['width'] * r['height']
    
    def test_rects(self):
        r1 = {'left': 1, 'right': 5, 'top': 3, 'width': 4, 'height': 4, 'bottom': 7}
        r2 = {'left': 3, 'right': 7, 'top': 1, 'width': 4, 'height': 4, 'bottom': 5}
        return [r1, r2]
        
        
    def test_intersection(self):
        #1 @ 1,3: 4x4
        #2 @ 3,1: 4x4
        r1, r2 = self.test_rects()
        r = self.intersection_rect(r1, r2)
        print("test:", r)
        assert r == {'left': 3, 'right': 5, 'top': 3, 'bottom': 5, 'width': 2, 'height': 2}
        
        r1 = {'left': 1, 'right': 2, 'top': 1, 'bottom': 2}
        r2 = {'left': 10, 'right': 11, 'top': 10, 'bottom': 11}
        r = self.intersection_rect(r1, r2)
        print("test:", r)
        
        
    def load(self):
        with open('day3.input') as f:
            self.lines = [ line.strip() for line in list(f) ]
            self.entries = [ self.parse_line(line) for line in self.lines ]

    
    def dimensions(self):
        width = 0
        height = 0
        for i in self.entries:
            width = max(i['right'], width)
            height = max(i['bottom'], height)
        return [width, height]
    
    def setup_fabric(self):
        width, height = self.dimensions()

        self.fabric = [[None for y in range(height + 1)] for x in range(width + 1)]
        self.overlaps = {}

    def paint_fabric(self, r):
        for x in range(r['left'], r['right']):
            for y in range(r['top'], r['bottom']):
                try:
                    existing = self.fabric[x][y]
                    if existing:
                        self.overlaps[existing] = 1
                        self.overlaps[r['index']] = 1

                    self.fabric[x][y] = r['index']
                except:
                    print("problem painting fabric x", x, "y", y)                    
    
    def get_non_overlapping(self):
        for r in self.entries:
            if r['index'] not in self.overlaps:
                return r['index']
        return None
        
        
    def run(self):
        self.load()

        self.test_intersection()
        total_area = 0
        
        self.setup_fabric()
        for r in self.entries:
            self.paint_fabric(r)
        
        return {
                'nonoverlapping': self.get_non_overlapping()}
                
        
if __name__ == "__main__":
    result = Day3b().run()
    print(result)
    # Answer: 119551

test: {'left': 3, 'right': 5, 'top': 3, 'bottom': 5, 'width': 2, 'height': 2}
test: {'left': 10, 'right': 2, 'top': 10, 'bottom': 2, 'width': -8, 'height': -8}
{'nonoverlapping': 1124}
